In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, roc_auc_score
import string
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import average_precision_score, precision_recall_curve
from imblearn.over_sampling import SMOTE
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, roc_auc_score
import string
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import average_precision_score, precision_recall_curve
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import cross_val_score, cross_val_predict
from nltk.stem.porter import *
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import GridSearchCV

In [ ]:
reviews_df = pd.read_csv("Hotel_Reviews.csv")
reviews_df.head(3)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968


In [ ]:
print("Duplicated rows before: ",reviews_df.duplicated().sum())
reviews_df.drop_duplicates(inplace=True)
print("Duplicated rows after: ",reviews_df.duplicated().sum())

Duplicated rows before:  526
Duplicated rows after:  0


In [ ]:
reviews_df.dropna(inplace=True) #Remember to DELETE!!!
reviews_df.isna().sum() #Remember to DELETE!!!

Hotel_Address                                 0
Additional_Number_of_Scoring                  0
Review_Date                                   0
Average_Score                                 0
Hotel_Name                                    0
Reviewer_Nationality                          0
Negative_Review                               0
Review_Total_Negative_Word_Counts             0
Total_Number_of_Reviews                       0
Positive_Review                               0
Review_Total_Positive_Word_Counts             0
Total_Number_of_Reviews_Reviewer_Has_Given    0
Reviewer_Score                                0
Tags                                          0
days_since_review                             0
lat                                           0
lng                                           0
dtype: int64

In [ ]:
print('No Positive values before:', end=" ")
print(reviews_df['Positive_Review'].value_counts().get('No Positive', 0))
print('No Negative values before:', end=" ")
print(reviews_df['Negative_Review'].value_counts().get('No Negative', 0))

reviews_df['Negative_Review']= reviews_df['Negative_Review'].replace("No Negative" , "")
reviews_df['Positive_Review']= reviews_df['Positive_Review'].replace("No Positive" , "")

reviews_df['Review'] = reviews_df['Negative_Review'].astype (str).str.cat (reviews_df['Positive_Review'], sep=' ')
reviews_df.head(5)

No Positive values before: 35695
No Negative values before: 126902


,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Review
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,I am so angry that i made this post available...
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,No real complaints the hotel was great great...
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Rooms are nice but for elderly a bit difficul...
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,My room was dirty and I was afraid to walk ba...
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,You When I booked with your company on line y...


In [ ]:
reviews_df_1 = reviews_df.loc[:, ['Review','Reviewer_Score']]
reviews_df_1.head(5)

,Review,Reviewer_Score
0,I am so angry that i made this post available...,2.9
1,No real complaints the hotel was great great...,7.5
2,Rooms are nice but for elderly a bit difficul...,7.1
3,My room was dirty and I was afraid to walk ba...,3.8
4,You When I booked with your company on line y...,6.7


In [ ]:
def abc(x):
  if x<=3.5:
    return 0
  elif (x<=6.5 and x>=5.5):
    return 1
  elif x==10:
    return 2
  else:
    return 3

reviews_df_1["Review_Type"] = reviews_df_1["Reviewer_Score"].apply(abc)
reviews_df_1.drop(reviews_df_1[reviews_df_1.Review_Type == 3].index, inplace=True)
reviews_df_1.head(5)

,Review,Reviewer_Score,Review_Type
0,I am so angry that i made this post available...,2.9,0
7,Apart from the price for the brekfast Everyth...,10.0,2
8,Even though the pictures show very clean room...,6.5,1
10,Nothing all great Rooms were stunningly dec...,10.0,2
11,6 30 AM started big noise workers loading woo...,5.8,1


In [ ]:
reviews_df_1['Review_Type'].value_counts()

2    115148
1     27587
0      6628
Name: Review_Type, dtype: int64

In [ ]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
stop_words = stopwords.words('english')
reviews_df_1['Review'] = reviews_df_1['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
reviews_df_1.head()

,Review,Reviewer_Score,Review_Type
0,I angry made post available via possible sites...,2.9,0
7,Apart price brekfast Everything good Good loca...,10.0,2
8,Even though pictures show clean rooms actual r...,6.5,1
10,Nothing great Rooms stunningly decorated reall...,10.0,2
11,6 30 AM started big noise workers loading wood...,5.8,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reviews_df_1['Review'], reviews_df_1['Review_Type'], test_size=0.3)

In [ ]:
'''
rus = RandomUnderSampler(sampling_strategy='all', random_state=42)

print(y_test.value_counts())

X_train_s, y_train_s = rus.fit_resample(X_train, y_train)

X_test_s, y_test_s = rus.fit_resample(X_test, y_test)

print(y_test_s.value_counts())
'''

"\nrus = RandomUnderSampler(sampling_strategy='all', random_state=42)\n\nprint(y_test.value_counts())\n\nX_train_s, y_train_s = rus.fit_resample(X_train, y_train)\n\nX_test_s, y_test_s = rus.fit_resample(X_test, y_test)\n\nprint(y_test_s.value_counts())\n"

In [ ]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

test_df
#total_data = pd.concat([train_df, test_df], axis=1)

,Review,Review_Type
443855,The fact stay longer Staff great room beautifu...,2
16077,No complaints Staff friendliness location faci...,2
147504,Having breakfast included price would perfect ...,2
200096,Concerts mean hotel captive audience charge no...,1
66402,The staff Rude incompetent staff,0
...,...,...
83552,Location great Room compact expected everythin...,2
451922,We went partners birthday extra effort make sp...,2
412705,Nothing square busy almost night adds atmosphe...,2
168860,I disappointed hotel It old need renovation Th...,1


In [ ]:
test_df.reset_index(inplace=True)
test_df

,index,Review,Review_Type
0,443855,The fact stay longer Staff great room beautifu...,2
1,16077,No complaints Staff friendliness location faci...,2
2,147504,Having breakfast included price would perfect ...,2
3,200096,Concerts mean hotel captive audience charge no...,1
4,66402,The staff Rude incompetent staff,0
...,...,...,...
44804,83552,Location great Room compact expected everythin...,2
44805,451922,We went partners birthday extra effort make sp...,2
44806,412705,Nothing square busy almost night adds atmosphe...,2
44807,168860,I disappointed hotel It old need renovation Th...,1


In [ ]:
test_df.columns

Index(['index', 'Review', 'Review_Type'], dtype='object')

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

#Load the pre-trained model
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
def roberta_polarity_scores(sentence):
    encoded_text = tokenizer(sentence, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict



In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
SIA=SentimentIntensityAnalyzer()

In [ ]:
res = {}
for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    try:
        text = row['Review']
        myid = row['index']
        vader_result = SIA.polarity_scores(text)
        vader_result_rename = {}
        for key, value in vader_result.items():
            vader_result_rename[f"vader_{key}"] = value
        roberta_result = roberta_polarity_scores(text)
        both = {**vader_result_rename, **roberta_result}
        res[myid] = both
    except RuntimeError:
        print(f'Broke at id {myid}')

results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'index'})
results_df = results_df.merge(test_df, how='left')


  0%|          | 0/44809 [00:00<?, ?it/s]

In [ ]:
results_df.head(5)

,index,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,Review,Review_Type
0,443855,0.000,0.391,0.609,0.8979,0.003532,0.042326,0.954142,The fact stay longer Staff great room beautifu...,2
1,16077,0.450,0.275,0.275,-0.2263,0.130588,0.654099,0.215313,No complaints Staff friendliness location faci...,2
2,147504,0.000,0.575,0.425,0.9595,0.003017,0.031512,0.965472,Having breakfast included price would perfect ...,2
3,200096,0.104,0.791,0.105,-0.2023,0.790007,0.182457,0.027536,Concerts mean hotel captive audience charge no...,1
4,66402,0.670,0.330,0.000,-0.7269,0.967604,0.029713,0.002684,The staff Rude incompetent staff,0


In [ ]:
results_df.to_csv("roberta.csv", encoding='utf-8')